In [ ]:
from dataset import *
from train import *

import csv

import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

In [ ]:
test_dir = '../../data/.train/.task148/data/test/images/'
test_csv = '../../data/.train/.task148/data/test/test.csv'

In [ ]:
distributions = {}
labels = ['10_콘크리트외벽', '20_조적외벽', '30_판넬외벽', '40_유리외벽', '50_기타외벽']
idxs = {29: ['0', '1', '2'], 31: ['0', '1'], 32: ['0', '1'], 36: ['', '부속건축물', '주건축물'], 
        44: ['', ' ', '강파이프구조', '경량철골구조', '공업화박판강구조(PEB)', '기타강구조', '기타구조', '기타조적구조', '기타철골철근콘크리트구조', '기타콘크리트구조', '목구조', '벽돌구조', '블록구조', '석구조', '시멘트블럭조', '일반목구조', '일반철골구조', '조립식판넬조', '조적구조', '철골구조', '철골철근콘크리트구조', '철골철근콘크리트합성구조', '철골콘크리트구조', '철근콘크리트구조', '콘크리트구조', '통나무구조', '트러스구조', '프리케스트콘크리트구조'], 
        50: ['', ' ', '(철근)콘크리트', '기와', '기타지붕', '슬레이트']}


labels = ['10_콘크리트외벽', '20_조적외벽', '30_판넬외벽', '40_유리외벽', '50_기타외벽']
test = []

with open(test_csv, 'r') as csvfile:
    for i, line in enumerate(csv.reader(csvfile)):
        if i != 0:
            ID = line[0]
            usage_list = []
            for idx in idxs:
                usage = [0]*len(idxs[idx])
                usage[idxs[idx].index(line[idx])] = 1
                usage_list += usage
            
            test.append((ID, tuple(usage_list)))

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5231, 0.5493, 0.5485], std=[0.2502, 0.2544, 0.2786])])

In [ ]:
test_dataset = Test_148(infos=test, transform=transform)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
img_features = models.resnet18()
img_features.fc = nn.Linear(512, len(labels))

meta_features = nn.Linear(len(usage_list), len(labels))
classifier = nn.Linear(2*len(labels), len(labels))

In [ ]:
img_features.load_state_dict(torch.load('./new_2_159_img.pt'))
meta_features.load_state_dict(torch.load('./new_2_159_meta.pt'))
classifier.load_state_dict(torch.load('./new_2_159_classifier.pt'))

In [ ]:
img_features.eval()
meta_features.eval()
classifier.eval()

In [ ]:
answers = []

with torch.no_grad():
    for i, (X, M) in enumerate(test_loader):
        img_output = img_features(X)
        meta_output = meta_features(M)
        output = classifier(torch.cat((img_output, meta_output), 1))
        output = torch.argmax(output, 1)
        answers += output.tolist()

In [ ]:
with open('./prediction.csv', 'a') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['ID', 'Target'])
    for i in range(len(answers)):
        ID = test[i][0]
        answer = answers[i]
        csv_writer.writerow([ID, labels[answer]])